Step 1: Gather information.

In [1]:
import os
import csv
import schedule
import time
import pandas as pd
from os import path
from csv import writer
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Set API parameters. 
load_dotenv()
API_KEY = os.getenv("API_KEY")
CHANNEL_ID = os.getenv("CHANNEL_ID")
youtube = build('youtube','v3', developerKey = API_KEY)

# Primary function to get statistics.
class ytStatsBase:
    def __init__(self, API_KEY, CHANNEL_ID, youtube):
        self.API_KEY = API_KEY
        self.CHANNEL_ID = CHANNEL_ID
        self.youtube = youtube

    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = CHANNEL_ID
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title']  , 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'] ,
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount']  , 
                result_channel_views = response['items'][i]['statistics']['viewCount']  , 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    def get_video_ids(self):
        playlist_id = ytStatsBase.get_channel_statistics(youtube)[0]['result_playlist_id']
        
        request = youtube.playlistItems().list(
            part = 'contentDetails' , 
            playlistId = playlist_id , 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails' , 
                    playlistId = playlist_id , 
                    maxResults = 50 , 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    def get_video_details(self):
        video_ids = ytStatsBase.get_video_ids(youtube)

        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics' , 
                id = ','.join(video_ids[i: i + 50]) ,
                #order = 'date' # Not possible or a feature at this time, 12Dec22.
            )
            response = request.execute()

            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title']  ,
                    result_video_id = video['id'] ,                
                    result_video_description = video['snippet']['description']  ,
                    result_video_upload_time = video['snippet']['publishedAt']  ,
                    result_video_views = video['statistics']['viewCount'] ,  
                    result_video_likes = video['statistics']['likeCount'] ,  
                    result_video_favorites = video['statistics']['favoriteCount'] ,  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)

        return all_data

# Call functions.
yt = ytStatsBase
channel_stats, video_details = yt.get_channel_statistics(youtube), yt.get_video_details(youtube)

# Move data to dataframes.
def create_data_frame(channel_stats, video_details):
    # Create data frames.
    initial_channel_data = pd.DataFrame(channel_stats, columns = [
        'result_playlist_id' ,
        'result_channel_name' , 
        'result_channel_views' ,
        'result_channel_subscribers' ,
        'result_channel_total_videos' 
        ])
    initial_video_data = pd.DataFrame(video_details, columns = [
        'result_video_id' ,
        'result_video_upload_time' , 
        'result_video_name' , 
        'result_video_description' , 
        'result_video_views' , 
        'result_video_likes' , 
        'result_video_comments' 
        ])
    return initial_channel_data, initial_video_data

dfc, dfv = create_data_frame(channel_stats, video_details)

Step 2: Write Information.

In [2]:
# Create base files. 
channel_name = channel_stats[0]['result_channel_name']

isExist1, isExist2 = os.path.exists('test_data_cha.csv'), os.path.exists('test_data_vid.csv')
if isExist1 is True:
    pass
elif isExist1 is False:
    dfc.to_csv(f'{channel_name}(channel_data).csv')

if isExist2 is True:
    pass
elif isExist2 is False:
    dfv.to_csv(f'{channel_name}(video_data).csv')

In [6]:
temp_data = dfv.head(1)
set_data = pd.read_csv(f'{channel_name}(channel_data).csv')

set_data.head(5)

,Unnamed: 0,result_playlist_id,result_channel_name,result_channel_views,result_channel_subscribers,result_channel_total_videos
0,0,UUzEy7pi3B7TIS9cn_sdKK9A,GrayStillPlays,2846254233,6180000,3013


In [7]:
# simply concatenate both dataframes
df = pd.concat([temp_data, dfv]).reset_index(drop = True)
df.head(5)

,result_video_id,result_video_upload_time,result_video_name,result_video_description,result_video_views,result_video_likes,result_video_comments
0,R_y9mrUX1UM,2022-12-14T22:30:02Z,Which ramps gives you the longest jump in GTA 5?,It's GTA 5.\n• Subscribe - http://bit.ly/1PG8z...,144669,8068,521
1,R_y9mrUX1UM,2022-12-14T22:30:02Z,Which ramps gives you the longest jump in GTA 5?,It's GTA 5.\n• Subscribe - http://bit.ly/1PG8z...,144669,8068,521
2,eGqDNDRYpA4,2022-12-13T22:02:24Z,"When you evolve dinosaurs for 9,999,999,999 years",It's Monster Catch Run.\n• Coffee - https://fl...,371132,12810,448
3,WIHknnyIVlU,2022-12-12T21:10:11Z,When you slice completely through earth,It's Slice It All!\n• Coffee - https://florida...,383754,13747,537
4,QlA8J3elpYg,2022-12-11T22:30:10Z,When a bullet shoots through the earth,It's GunSpin.\n• Coffee - https://floridamanco...,710634,21260,1134
